In [18]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [23]:
import psycopg2
import pandas as pd
import numpy as np
import requests

In [24]:
#Conexión Lake Cencommerce
# Update the following with your Redshift connection details
redshift_host = 'rs-cl-sbx.csmzzf6sb3pd.us-east-1.redshift.amazonaws.com'
redshift_ip = '172.23.55.223'
redshift_port = '5439'
redshift_db = 'sandbox_corp'
redshift_user = 'elias_salehmuse'
redshift_password = 'vtMPcp2u3B3q'

def execute_query(query):
    try:
        # Establish a connection to Redshift
        conn = psycopg2.connect(
            host=redshift_host,
            port=redshift_port,
            database=redshift_db,
            user=redshift_user,
            password=redshift_password
        )

        # Create a cursor
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)

        # Fetch all results
        result = cursor.fetchall()

        # Get column names
        columns = [desc[0] for desc in cursor.description]

        # Create a Pandas DataFrame
        df = pd.DataFrame(result, columns=columns)

        # Close the cursor and connection
        cursor.close()
        conn.close()

        return df

    except Exception as e:
        print(f"Error: {e}")

In [25]:
#Conexión BDD AUX Delivery
# Update the following with your Redshift connection details
redshift_host1 = 'daas-report-bagging.crrrlku8fkeu.us-east-1.rds.amazonaws.com'
redshift_ip1= '172.23.55.223'
redshift_port1 = '5432'
redshift_db1 = 'daas_report_bagging'
redshift_user1 = 'elias_saleh_db'
redshift_password1 = 'yCWkGK9H7qO6JB6LLhOu3'

def execute_query1(query1):
    try:
        # Establish a connection to Redshift
        conn1= psycopg2.connect(
            host=redshift_host1,
            port=redshift_port1,
            database=redshift_db1,
            user=redshift_user1,
            password=redshift_password1
        )

        # Create a cursor
        cursor1 = conn1.cursor()

        # Execute the query
        cursor1.execute(query1)

        # Fetch all results
        result1 = cursor1.fetchall()

        # Get column names
        columns1 = [desc[0] for desc in cursor1.description]

        # Create a Pandas DataFrame
        df1 = pd.DataFrame(result1, columns=columns1)

        # Close the cursor and connection
        cursor1.close()
        conn1.close()

        return df1

    except Exception as e:
        print(f"Error: {e}")

In [26]:
#Query Filters

fecha_compromiso_inicial = '2024-02-01 0:00:00'
fecha_compromiso_final = '2024-02-03 23:59:59'
negocio = 'Jumbo' #Jumbo, Santa Isabel, Spid (Chile)


In [27]:
# Example query lake
query = f"""
select
    *
from
    (
        select
            --ExternalSystemOrderId
            dispatch_order.external_system_order_id as orden_compra,
            --Courier
            hd_delivery_dispatch_raw_prod_spec.courier.name as sistema,
            --ExternalDriverId
            hd_delivery_dispatch_raw_prod.dispatch_order.external_driver_id as codigo_conductor,
            --Codigo País
            hd_delivery_dispatch_raw_prod_spec.country.code as codigo_pais,
            --Courier
            case
                when sistema = 'Beetrack' then tags1.value --Tags1 Table
                when sistema = 'Home Delivery Chile' then provider.name --Provider Table
                else sistema --Courier
            end as proveedor,
            --LicencePlate
            case
                when sistema = 'Beetrack' then tags2.value --Tags2 Table
                when sistema = 'Home Delivery Chile' then hd_delivery_tms_raw_prod_spec.vehicle.licence_plate --Vehicle Licence Plate
                else sistema --Courier
            end as patente,
            --driverName
            case
                when sistema = 'Beetrack' then hd_delivery_dispatch_raw_prod.vehicle_driver.full_name --driverName from VehicleDriver
                else hd_delivery_tms_raw_prod_spec.driver.full_name --driverName from driver
            end as nombre_driver,
            --RetailChainName
            retail_chain.name as negocio,
            --StoreCode
            known_source.external_id as codigo_tienda,
            --StoreName
            known_source.name as nombre_tienda,
            --EventDate from TrackingEvents BD
            tmp_te."event-date" :: timestamp as fecha_evento,
            --CommitmentDate
            case
                when codigo_pais = 'PE' then date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                )
                when codigo_pais = 'CO' then date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                )
                when codigo_pais = 'AR' then date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                )
                when codigo_pais = 'BR' then date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                )
                else date(dispatch_order.eta_end_date :: timestamp)
            end as fecha_compromiso,
            --LastStatusDesc
            pack2.last_status_desc as nombre_estado,
            --LastSubStatusDesc
            pack2.last_sub_status_desc as nombre_sub_estado,
            --SubAdminZone2 Name
            saz2_destination.name as comuna_destino,
            --DestinationLatitude
            destination.latitude as latitude,
            --DestinationLogitude
            destination.longitude as longitude,
            --OnTime
            case
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'PE'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp - INTERVAL '1 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) < '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'PE'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp - INTERVAL '2 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) >= '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'CO'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp - INTERVAL '1 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) < '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'CO'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp - INTERVAL '2 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-5'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) >= '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'AR'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp + INTERVAL '1 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) < '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'AR'
                and (
                    DATE_TRUNC('minute', tmp_te."event-date" :: timestamp) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) >= '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'BR'
                and (
                    DATE_TRUNC(
                        'minute',
                        tmp_te."event-date" :: timestamp + INTERVAL '1 hour'
                    ) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) < '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'BR'
                and (
                    DATE_TRUNC('minute', tmp_te."event-date" :: timestamp) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-3'
                    )
                )
                and date(
                    dispatch_order.eta_end_date :: timestamp AT TIME ZONE 'UTC-4'
                ) >= '09/03/2023' then 1
                when hd_delivery_dispatch_raw_prod_spec.country.code = 'CL'
                and (
                    DATE_TRUNC('minute', tmp_te."event-date" :: timestamp) <= DATE_TRUNC(
                        'minute',
                        dispatch_order.eta_end_date :: timestamp
                    )
                ) then 1
                else 0
            end as on_time,
            --OrderKey
            dispatch_order.external_system_order_id || '|' || max(dispatch_order.updated_at :: timestamp) as order_key
        from
            --DispatchOrder
            hd_delivery_dispatch_raw_prod.dispatch_order
            left join (
                select
                    *
                from
                    (
                        select
                            pack.dispatch_order_id,
                            pack.tracking,
                            pack.last_status_code,
                            pack.last_status_desc,
                            pack.last_sub_status_code,
                            pack.last_sub_status_desc,
                            pack.tracking || '|' || max(pack.updated_at :: timestamp) as pack_key
                        from
                            hd_delivery_dispatch_raw_prod.pack --where pack.tracking = 'v500140173spid-01'
                        group by
                            1,
                            2,
                            3,
                            4,
                            5,
                            6
                    ) as tempo
                where
                    tempo.pack_key in (
                        select
                            tmp00.pack_key
                        from
                            (
                                select
                                    pack.tracking,
                                    pack.tracking || '|' || max(pack.updated_at :: timestamp) as pack_key
                                from
                                    hd_delivery_dispatch_raw_prod.pack --where pack.tracking = 'v500140173spid-01'
                                group by
                                    1
                            ) tmp00
                    )
            ) as pack2 on pack2.dispatch_order_id = dispatch_order.id --TODO
            left join hd_delivery_dispatch_raw_prod.destination on destination.id = dispatch_order.destination_id --Destination
            left join hd_delivery_dispatch_raw_prod_spec.sub_admin_zone_1 saz1_destination on saz1_destination.id = destination.sub_admin_zone_1_id --SubAdminZone1 Destination
            left join hd_delivery_dispatch_raw_prod_spec.sub_admin_zone_2 saz2_destination on saz2_destination.id = destination.sub_admin_zone_2_id --SubAdminZone2 Destination
            left join hd_delivery_dispatch_raw_prod_spec.known_source on known_source.id = dispatch_order.owner_known_source_id --Known Source
            left join hd_delivery_dispatch_raw_prod_spec.retail_chain on hd_delivery_dispatch_raw_prod_spec.retail_chain.id = hd_delivery_dispatch_raw_prod.dispatch_order.retail_chain_id --Retail chain
            left join hd_delivery_dispatch_raw_prod_spec.country on hd_delivery_dispatch_raw_prod_spec.country.id = hd_delivery_dispatch_raw_prod_spec.retail_chain.country_id --Country
            left join hd_delivery_dispatch_raw_prod_spec.dispatch_type on hd_delivery_dispatch_raw_prod.dispatch_order.dispatch_type_id = hd_delivery_dispatch_raw_prod_spec.dispatch_type.id --Dispatch Type
            left join hd_delivery_dispatch_raw_prod_spec.courier on hd_delivery_dispatch_raw_prod_spec.courier.id = hd_delivery_dispatch_raw_prod.dispatch_order.courier_id --Courier
            left join hd_delivery_tms_raw_prod_spec.driver on hd_delivery_tms_raw_prod_spec.driver.id = hd_delivery_dispatch_raw_prod.dispatch_order.external_driver_id --Driver TMS
            left join hd_delivery_tms_raw_prod_spec.vehicle on hd_delivery_tms_raw_prod_spec.vehicle.id = hd_delivery_tms_raw_prod_spec.driver.vehicle_id --Vehicle TMS
            left join hd_delivery_dispatch_raw_prod_spec.provider on hd_delivery_tms_raw_prod_spec.driver.provider_id = hd_delivery_dispatch_raw_prod_spec.provider.id --Provider from TMS
            left join (
                select
                    te.external_system_order_id,
                    te."event-date",
                    ps.description
                from
                    hd_delivery_tracking_raw_prod.tracking_event te
                    left join hd_delivery_tracking_raw_prod_spec.pack_status ps on te.pack_status_id = ps.id
                where
                    te.id in (
                        select
                            max(te0.id) as id0
                        from
                            hd_delivery_tracking_raw_prod.tracking_event te0
                        group by
                            te0.external_system_order_id
                    )
            ) as tmp_te on dispatch_order.external_system_order_id = tmp_te.external_system_order_id -- Tracking Events, LastStatus
            left join (
                select
                    dispatch_order_id,
                    name,
                    value,
                    id
                from
                    hd_delivery_dispatch_raw_prod."tag"
                where
                    1 = 1
                    and (
                        lower(hd_delivery_dispatch_raw_prod."tag".name) like '%transportes%'
                    ) --and dispatch_order_id = 15258246
                    --group by dispatch_order_id,name,value
                    and hd_delivery_dispatch_raw_prod."tag".id in (
                        select
                            aux0.maxid
                        from
                            (
                                select
                                    dispatch_order_id,
                                    name,
                                    max (id) as maxid
                                from
                                    hd_delivery_dispatch_raw_prod."tag"
                                where
                                    1 = 1 --and dispatch_order_id = 14496340
                                    and (
                                        lower(hd_delivery_dispatch_raw_prod."tag".name) like '%transportes%'
                                    )
                                group by
                                    dispatch_order_id,
                                    name
                            ) as aux0
                    )
            ) as tags1 on dispatch_order.id = tags1.dispatch_order_id --Tags
            left join (
                select
                    dispatch_order_id,
                    name,
                    value,
                    id
                from
                    hd_delivery_dispatch_raw_prod."tag"
                where
                    1 = 1
                    and (
                        hd_delivery_dispatch_raw_prod."tag".name = 'truck_identifier'
                    ) --and dispatch_order_id = 15258246
                    --group by dispatch_order_id,name,value
                    and hd_delivery_dispatch_raw_prod."tag".id in (
                        select
                            aux1.maxid
                        from
                            (
                                select
                                    dispatch_order_id,
                                    name,
                                    max (id) as maxid
                                from
                                    hd_delivery_dispatch_raw_prod."tag"
                                where
                                    1 = 1 --and dispatch_order_id = 14496340
                                    and (
                                        hd_delivery_dispatch_raw_prod."tag".name = 'truck_identifier'
                                    )
                                group by
                                    dispatch_order_id,
                                    name
                            ) as aux1
                    )
            ) as tags2 on dispatch_order.id = tags2.dispatch_order_id --Tags2
            left join hd_delivery_dispatch_raw_prod.vehicle_driver on hd_delivery_dispatch_raw_prod.vehicle_driver.id = hd_delivery_dispatch_raw_prod.dispatch_order.vehicle_driver_id --VehicleDriver
        where
            1 = 1
            and dispatch_order.retail_chain_id in (1, 4, 9) --RetailChainId
            and pack2.last_status_code not in ('10', '1002') --LastStatusCode
            and pack2.last_status_code in ('09', '02', '04', '03') --LastStatusCode
        group by
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11,
            12,
            13,
            14,
            15,
            16,
            17,
            18 --[[and {{created_at}}]]
            --[[and {{update_at}}]]
            --[[and {{nombre_tienda}}]]
            --[[and {{codigo_tienda}}]]
            --[[and {{orden_compra}}]]
            --[[and {{estado}}]]
            --[[and {{subestado}}]]
            --[[and {{patente}}]]
            --[[and {{nombre_conductor}}]]
            --[[and {{negocio}}]]
            --[[and {{courier}}]]
    ) as tmp
where
    1 = 1
    and tmp.order_key in (
        select
            tmp000.order_key
        from
            (
                select
                    dispatch_order.external_system_order_id,
                    dispatch_order.external_system_order_id || '|' || max(dispatch_order.updated_at :: timestamp) as order_key
                from
                    hd_delivery_dispatch_raw_prod.dispatch_order --where dispatch_order.external_system_order_id = 'v500140173spid-01'
                group by
                    1
            ) as tmp000
    )
    and tmp.negocio = '{negocio}'
    --and tmp.proveedor <> 'None'
    --and tmp.nombre_driver <> 'None'
    and tmp.fecha_compromiso >= '{fecha_compromiso_inicial}'
    and tmp.fecha_compromiso <= '{fecha_compromiso_final}' --[[and  tmp.fecha_evento>= ({{updated_at1}}::date || ' 0:00:00')::timestamp]]
    --[[and  tmp.fecha_evento<= ({{updated_at2}}::date || ' 23:59:59')::timestamp]]
    --[[and  tmp.fecha_compromiso>= ({{eta_1}}::date || ' 0:00:00')::timestamp]]
    --[[and  tmp.fecha_compromiso<= ({{eta_2}}::date || ' 23:59:59')::timestamp]]
    --[[and tmp.proveedor = {{proveedor}}]]
"""



In [28]:
#Query Aux

query1= f"""
select order_id,
sum(case 
	when ROUND(quantity::numeric, 0) = quantity THEN quantity::integer
    ELSE 1
end) as bultos

from daas_report_bagging.report_bagging.item
group by order_id
"""

**EJECUTAR 1**

In [29]:
# Execute the query and get the result in a DataFrame
result_df = execute_query(query)


In [30]:
result_df

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key
0,v122093033jmch-01,Beetrack,,CL,Touch,TOUCH-PRHR55,Alexaris Yinneth Cañizalez Cortez,Jumbo,J502,JUMBO KENNEDY,2024-02-01 10:09:51,2024-02-01,Entregado,Entregado a Domicilio,Las Condes,-33.411898400000000,-70.534310000000000,1,v122093033jmch-01|2024-02-01 10:10:57.902
1,v122332286jmch-01,Beetrack,,CL,Boosmap,BOOS-RVJF35,CARLOTA MILAGRO BRACAMONTE MARTINEZ,Jumbo,J403,JUMBO MAIPU,2024-02-03 17:30:39,2024-02-03,Entregado,Entregado a Domicilio,Maipú,-33.527309600000000,-70.761931600000000,1,v122332286jmch-01|2024-02-03 17:31:59.719
2,v122246280jmch-01,Beetrack,,CL,Foxer,FOXER-HTCC96,MANOLITA FERNÁNDEZ HERNANDEZ,Jumbo,J521,JUMBO LA SERENA,2024-02-03 11:07:23,2024-02-03,Entregado,Entregado a Domicilio,Coquimbo,-30.085143472136200,-71.369544155895700,1,v122246280jmch-01|2024-02-03 11:08:43.041
3,v122315879jmch-01,Beetrack,,CL,Foxer,FOXER-PLFZ51,juan eduardo moya alarcon,Jumbo,J534,JUMBO ANTOFAGASTA,2024-02-03 14:45:17,2024-02-03,Recogido,Entregado a Domicilio,Antofagasta,-23.527333800000000,-70.396862500000000,1,v122315879jmch-01|2024-02-03 15:53:41.522
4,v122317437jmch-01,Beetrack,,CL,Foxer,FOXER-CHLL99,Julio Antonio Pizarro Leyton,Jumbo,J534,JUMBO ANTOFAGASTA,2024-02-03 14:47:28,2024-02-03,Entregado,Entregado a Domicilio,Antofagasta,-23.667771300000000,-70.405912700000000,1,v122317437jmch-01|2024-02-03 14:48:53.205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55498,v122268505jmch-01,Beetrack,,CL,Touch,TOUCH-SVVK11,Juan José Cabrera Pereira,Jumbo,J502,JUMBO KENNEDY,2024-02-03 09:57:02,2024-02-03,Entregado,Entregado a Domicilio,Vitacura,-33.398139000000000,-70.592261000000000,1,v122268505jmch-01|2024-02-03 09:59:14.183
55499,v122299026jmch-01,Beetrack,,CL,Boosmap,BOOS-VFKB29,KARIN LUZ FUENTES CARREÑO,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-03 11:50:45,2024-02-03,Entregado,Entregado a Domicilio,San Bernardo,-33.623297700000000,-70.710229100000000,1,v122299026jmch-01|2024-02-03 11:53:17.082
55500,v122171820jmch-01,Beetrack,,CL,Touch,TOUCH-SFZS59,Jose Bernardo Flores,Jumbo,J502,JUMBO KENNEDY,2024-02-02 11:06:23,2024-02-02,Entregado,Entregado a Domicilio,Vitacura,-33.405333000000000,-70.596109000000000,1,v122171820jmch-01|2024-02-02 11:07:42.582
55501,v122173223jmch-01,Beetrack,,CL,Kowski,KOWSKI-GFCB74,MARCELA AGUILERA LIZAMA,Jumbo,J511,JUMBO COSTANERA,2024-02-02 09:46:16,2024-02-02,Entregado,Entregado a Domicilio,Recoleta,-33.398466100000000,-70.654638700000000,1,v122173223jmch-01|2024-02-02 09:47:23.295


**EJECUTAR 2**

In [31]:
result_df1 = execute_query1(query1)

In [32]:
#Merges with another data sources

**EJECUTAR 3**

In [33]:
# Display the DataFrame

result_df

#System Filter
#result_df_1= result_df[result_df['sistema']=='Home Delivery Chile']

#Provider Filter
#result_df_2= result_df[result_df['proveedor']=='Boosmap Chile']
#result_df_2

#Store Code Filter
#result_df_3= result_df[result_df['codigo_tienda']=='J762']
#result_df_3



,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key
0,v122093033jmch-01,Beetrack,,CL,Touch,TOUCH-PRHR55,Alexaris Yinneth Cañizalez Cortez,Jumbo,J502,JUMBO KENNEDY,2024-02-01 10:09:51,2024-02-01,Entregado,Entregado a Domicilio,Las Condes,-33.411898400000000,-70.534310000000000,1,v122093033jmch-01|2024-02-01 10:10:57.902
1,v122332286jmch-01,Beetrack,,CL,Boosmap,BOOS-RVJF35,CARLOTA MILAGRO BRACAMONTE MARTINEZ,Jumbo,J403,JUMBO MAIPU,2024-02-03 17:30:39,2024-02-03,Entregado,Entregado a Domicilio,Maipú,-33.527309600000000,-70.761931600000000,1,v122332286jmch-01|2024-02-03 17:31:59.719
2,v122246280jmch-01,Beetrack,,CL,Foxer,FOXER-HTCC96,MANOLITA FERNÁNDEZ HERNANDEZ,Jumbo,J521,JUMBO LA SERENA,2024-02-03 11:07:23,2024-02-03,Entregado,Entregado a Domicilio,Coquimbo,-30.085143472136200,-71.369544155895700,1,v122246280jmch-01|2024-02-03 11:08:43.041
3,v122315879jmch-01,Beetrack,,CL,Foxer,FOXER-PLFZ51,juan eduardo moya alarcon,Jumbo,J534,JUMBO ANTOFAGASTA,2024-02-03 14:45:17,2024-02-03,Recogido,Entregado a Domicilio,Antofagasta,-23.527333800000000,-70.396862500000000,1,v122315879jmch-01|2024-02-03 15:53:41.522
4,v122317437jmch-01,Beetrack,,CL,Foxer,FOXER-CHLL99,Julio Antonio Pizarro Leyton,Jumbo,J534,JUMBO ANTOFAGASTA,2024-02-03 14:47:28,2024-02-03,Entregado,Entregado a Domicilio,Antofagasta,-23.667771300000000,-70.405912700000000,1,v122317437jmch-01|2024-02-03 14:48:53.205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55498,v122268505jmch-01,Beetrack,,CL,Touch,TOUCH-SVVK11,Juan José Cabrera Pereira,Jumbo,J502,JUMBO KENNEDY,2024-02-03 09:57:02,2024-02-03,Entregado,Entregado a Domicilio,Vitacura,-33.398139000000000,-70.592261000000000,1,v122268505jmch-01|2024-02-03 09:59:14.183
55499,v122299026jmch-01,Beetrack,,CL,Boosmap,BOOS-VFKB29,KARIN LUZ FUENTES CARREÑO,Jumbo,J843,JUMBO SAN BERNARDO,2024-02-03 11:50:45,2024-02-03,Entregado,Entregado a Domicilio,San Bernardo,-33.623297700000000,-70.710229100000000,1,v122299026jmch-01|2024-02-03 11:53:17.082
55500,v122171820jmch-01,Beetrack,,CL,Touch,TOUCH-SFZS59,Jose Bernardo Flores,Jumbo,J502,JUMBO KENNEDY,2024-02-02 11:06:23,2024-02-02,Entregado,Entregado a Domicilio,Vitacura,-33.405333000000000,-70.596109000000000,1,v122171820jmch-01|2024-02-02 11:07:42.582
55501,v122173223jmch-01,Beetrack,,CL,Kowski,KOWSKI-GFCB74,MARCELA AGUILERA LIZAMA,Jumbo,J511,JUMBO COSTANERA,2024-02-02 09:46:16,2024-02-02,Entregado,Entregado a Domicilio,Recoleta,-33.398466100000000,-70.654638700000000,1,v122173223jmch-01|2024-02-02 09:47:23.295


In [34]:
#System Filter
result_df_4= result_df[result_df['orden_compra']=='v122157200jmch-01']


result_df_4

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key


**EJECUTAR 4**

In [35]:
result_df1= result_df1.rename(columns={'order_id': 'orden_compra'})



result_df1



,orden_compra,bultos
0,v100042382jmch-01,4
1,v100581499jmch-01,4
2,v102026913jmch-01,5
3,v102992382jmch-01,8
4,v103021946jmch-01,6
...,...,...
2006846,v99468007jmch-01,7
2006847,v99471841jmch-01,3
2006848,v99481692jmch-01,2
2006849,v99529141jmch-01,4


**EJECUTAR 5**

In [55]:
result_cruce = pd.merge(result_df, result_df1, on='orden_compra', how='left')

result_cruce['final_size'] = ''
result_cruce.loc[(result_cruce['bultos']>0)&(result_cruce['bultos']<=5),'final_size']='MP'
result_cruce.loc[(result_cruce['bultos']>5)&(result_cruce['bultos']<=12),'final_size']='P'
result_cruce.loc[(result_cruce['bultos']>12)&(result_cruce['bultos']<=20),'final_size']='M'
result_cruce.loc[(result_cruce['bultos']>20),'final_size']='G'

result_cruce

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,...,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size
7778,v121932843jmch-01,Home Delivery Chile,,CL,None,None,None,Jumbo,J507,JUMBO PTO. MONTT,...,2024-02-01,Entregado,Entregado a Domicilio,Puerto Octay,-40.895363605110900,-72.391498998828100,1,v121932843jmch-01|2024-02-01 20:41:07.719,11.0,P


**EJECUTAR 6**

In [37]:
import numpy as np
def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'suburb' in data['address']:
            return data['address']['suburb']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            return "Commune not found" 
    else:
        return "Error fetching data"  # Return error message if request was not successful

def get_communes(row):
    if pd.isna(row['comuna_destino']):  # Check if commune column is empty
        commune = get_commune(row['latitude'], row['longitude'])
        return commune
    else:
        return row['comuna_destino']
    

# Apply get_communes function to each row
result_cruce['comuna_destino'] = result_cruce.apply(get_communes, axis=1)



# Print the resulting DataFrame
result_cruce

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,...,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size
0,v122093033jmch-01,Beetrack,,CL,Touch,TOUCH-PRHR55,Alexaris Yinneth Cañizalez Cortez,Jumbo,J502,JUMBO KENNEDY,...,2024-02-01,Entregado,Entregado a Domicilio,Las Condes,-33.411898400000000,-70.534310000000000,1,v122093033jmch-01|2024-02-01 10:10:57.902,4.0,MP
1,v122332286jmch-01,Beetrack,,CL,Boosmap,BOOS-RVJF35,CARLOTA MILAGRO BRACAMONTE MARTINEZ,Jumbo,J403,JUMBO MAIPU,...,2024-02-03,Entregado,Entregado a Domicilio,Maipú,-33.527309600000000,-70.761931600000000,1,v122332286jmch-01|2024-02-03 17:31:59.719,18.0,M
2,v122246280jmch-01,Beetrack,,CL,Foxer,FOXER-HTCC96,MANOLITA FERNÁNDEZ HERNANDEZ,Jumbo,J521,JUMBO LA SERENA,...,2024-02-03,Entregado,Entregado a Domicilio,Coquimbo,-30.085143472136200,-71.369544155895700,1,v122246280jmch-01|2024-02-03 11:08:43.041,20.0,M
3,v122315879jmch-01,Beetrack,,CL,Foxer,FOXER-PLFZ51,juan eduardo moya alarcon,Jumbo,J534,JUMBO ANTOFAGASTA,...,2024-02-03,Recogido,Entregado a Domicilio,Antofagasta,-23.527333800000000,-70.396862500000000,1,v122315879jmch-01|2024-02-03 15:53:41.522,6.0,P
4,v122317437jmch-01,Beetrack,,CL,Foxer,FOXER-CHLL99,Julio Antonio Pizarro Leyton,Jumbo,J534,JUMBO ANTOFAGASTA,...,2024-02-03,Entregado,Entregado a Domicilio,Antofagasta,-23.667771300000000,-70.405912700000000,1,v122317437jmch-01|2024-02-03 14:48:53.205,5.0,MP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55498,v122268505jmch-01,Beetrack,,CL,Touch,TOUCH-SVVK11,Juan José Cabrera Pereira,Jumbo,J502,JUMBO KENNEDY,...,2024-02-03,Entregado,Entregado a Domicilio,Vitacura,-33.398139000000000,-70.592261000000000,1,v122268505jmch-01|2024-02-03 09:59:14.183,14.0,M
55499,v122299026jmch-01,Beetrack,,CL,Boosmap,BOOS-VFKB29,KARIN LUZ FUENTES CARREÑO,Jumbo,J843,JUMBO SAN BERNARDO,...,2024-02-03,Entregado,Entregado a Domicilio,San Bernardo,-33.623297700000000,-70.710229100000000,1,v122299026jmch-01|2024-02-03 11:53:17.082,6.0,P
55500,v122171820jmch-01,Beetrack,,CL,Touch,TOUCH-SFZS59,Jose Bernardo Flores,Jumbo,J502,JUMBO KENNEDY,...,2024-02-02,Entregado,Entregado a Domicilio,Vitacura,-33.405333000000000,-70.596109000000000,1,v122171820jmch-01|2024-02-02 11:07:42.582,22.0,G
55501,v122173223jmch-01,Beetrack,,CL,Kowski,KOWSKI-GFCB74,MARCELA AGUILERA LIZAMA,Jumbo,J511,JUMBO COSTANERA,...,2024-02-02,Entregado,Entregado a Domicilio,Recoleta,-33.398466100000000,-70.654638700000000,1,v122173223jmch-01|2024-02-02 09:47:23.295,1.0,MP


**EJECUTAR 7**

In [56]:
result_cruce['provider_key'] = result_cruce['codigo_tienda']+"|"+result_cruce['proveedor']+'|'+result_cruce['comuna_destino']+'|'+result_cruce['final_size']

result_cruce

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,...,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size,provider_key
7778,v121932843jmch-01,Home Delivery Chile,,CL,None,None,None,Jumbo,J507,JUMBO PTO. MONTT,...,Entregado,Entregado a Domicilio,Puerto Octay,-40.895363605110900,-72.391498998828100,1,v121932843jmch-01|2024-02-01 20:41:07.719,11.0,P,NaN


In [41]:
result_cruce_2=result_cruce[result_cruce['orden_compra']=='v121932843jmch-01']

result_cruce_2

,orden_compra,sistema,codigo_conductor,codigo_pais,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,...,nombre_estado,nombre_sub_estado,comuna_destino,latitude,longitude,on_time,order_key,bultos,final_size,provider_key
7778,v121932843jmch-01,Home Delivery Chile,,CL,None,None,None,Jumbo,J507,JUMBO PTO. MONTT,...,Entregado,Entregado a Domicilio,Puerto Octay,-40.895363605110900,-72.391498998828100,1,v121932843jmch-01|2024-02-01 20:41:07.719,11.0,P,NaN


**EJECUTAR 8**

In [43]:
#Open Pricing
import os
pricing_excel_file_path = os.path.join(os.getcwd(), "Tarifario.xlsx")

df_pricing = pd.read_excel(pricing_excel_file_path)

df_pricing


,Business,Store_ID,Provider,Destination,Size,Cost,HD
0,Santa Isabel,N841,Transporte Campillay,Copiapó,MP,4900,NO
1,Santa Isabel,N841,Transporte Campillay,Caldera,MP,4900,NO
2,Santa Isabel,N841,Transporte Campillay,Tierra Amarilla,MP,4900,NO
3,Santa Isabel,N841,Transporte Campillay,Huasco,MP,4900,NO
4,Santa Isabel,N841,Transporte Campillay,Vallenar,MP,4900,NO
...,...,...,...,...,...,...,...
19736,Jumbo,J532,ZUBALE,Quillón,P,16331,NO
19737,Jumbo,J532,ZUBALE,Quillón,M,17147,NO
19738,Jumbo,J532,ZUBALE,Quillón,G,21169,NO
19739,Jumbo,J532,ZUBALE,Quillón,Extra G,23909,NO


**EJECUTAR 9**

In [44]:
df_pricing['provider_key'] = df_pricing['Store_ID'].astype(str)+"|"+df_pricing['Provider']+'|'+df_pricing['Destination']+'|'+df_pricing['Size']




df_pricing

,Business,Store_ID,Provider,Destination,Size,Cost,HD,provider_key
0,Santa Isabel,N841,Transporte Campillay,Copiapó,MP,4900,NO,N841|Transporte Campillay|Copiapó|MP
1,Santa Isabel,N841,Transporte Campillay,Caldera,MP,4900,NO,N841|Transporte Campillay|Caldera|MP
2,Santa Isabel,N841,Transporte Campillay,Tierra Amarilla,MP,4900,NO,N841|Transporte Campillay|Tierra Amarilla|MP
3,Santa Isabel,N841,Transporte Campillay,Huasco,MP,4900,NO,N841|Transporte Campillay|Huasco|MP
4,Santa Isabel,N841,Transporte Campillay,Vallenar,MP,4900,NO,N841|Transporte Campillay|Vallenar|MP
...,...,...,...,...,...,...,...,...
19736,Jumbo,J532,ZUBALE,Quillón,P,16331,NO,J532|ZUBALE|Quillón|P
19737,Jumbo,J532,ZUBALE,Quillón,M,17147,NO,J532|ZUBALE|Quillón|M
19738,Jumbo,J532,ZUBALE,Quillón,G,21169,NO,J532|ZUBALE|Quillón|G
19739,Jumbo,J532,ZUBALE,Quillón,Extra G,23909,NO,J532|ZUBALE|Quillón|Extra G


In [45]:
df_pricing1=df_pricing[df_pricing['provider_key']=='J504|ZUBALE|Las Cabras|M']

df_pricing1

,Business,Store_ID,Provider,Destination,Size,Cost,HD,provider_key
17236,Jumbo,J504,ZUBALE,Las Cabras,M,25903,NO,J504|ZUBALE|Las Cabras|M


**EJECUTAR 10**

In [73]:
#Deleting rows without provider_key
result_cruce_with_provider_key = result_cruce.dropna(subset=['provider_key'])
#Rows without provider_key
result_cruce_nan = result_cruce[result_cruce['provider_key'].isna()]

result_pricing = pd.merge(result_cruce_with_provider_key, df_pricing, on='provider_key', how= 'left')

#Logic for provider_key = NaN
if not result_cruce_nan.empty:
    # Concat rows
    result_pricing = pd.concat([result_pricing, result_cruce_nan], ignore_index=True)

result_pricing = result_pricing[['orden_compra', 'sistema', 'codigo_conductor','proveedor','patente','nombre_driver','negocio','codigo_tienda','nombre_tienda','fecha_evento','fecha_compromiso','nombre_estado','nombre_sub_estado','comuna_destino','on_time','bultos','final_size','provider_key','Cost']]

result_pricing['final_cost'] = result_pricing.apply(lambda row: row['Cost'] * 0.5 if row['nombre_estado'] == 'Entregado Parcial' else (row['Cost'] * 0.5 if row['nombre_estado'] == 'No Entregado' else row['Cost']), axis=1)
#result_pricing
#result_pricing[result_pricing['orden_compra']=='v121932843jmch-01']
result_pricing[result_pricing['orden_compra']=='v121932843jmch-01']

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost
54807,v121932843jmch-01,Home Delivery Chile,,None,None,None,Jumbo,J507,JUMBO PTO. MONTT,2024-02-01 20:41:07.063,2024-02-01,Entregado,Entregado a Domicilio,Puerto Octay,1,11.0,P,NaN,NaN,NaN


In [74]:
result_pricing_2=result_pricing[result_pricing['codigo_tienda']=='J514']
result_pricing_2

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost
8,v122250776jmch-01,Beetrack,,Boosmap,BOOS-SJGL74,Angel Doza Cruz,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 16:59:44,2024-02-02,Entregado,Entregado a Domicilio,Lo Barnechea,1,2.0,MP,J514|Boosmap|Lo Barnechea|MP,NaN,NaN
25,v122275988jmch-01,Beetrack,,Boosmap,BOOS-RHWK77,Ricardo Velasco Morales,Jumbo,J514,JUMBO LA DEHESA,2024-02-03 10:06:35,2024-02-03,Entregado,Entregado a Domicilio,Colina,1,5.0,MP,J514|Boosmap|Colina|MP,NaN,NaN
28,v122305134jmch-01,Beetrack,,Boosmap,Boos-Lxhl27,Alvaro David SOJO SALAVARRIA,Jumbo,J514,JUMBO LA DEHESA,2024-02-03 12:06:50,2024-02-03,Recogido,Entregado a Domicilio,Lo Barnechea,1,6.0,P,J514|Boosmap|Lo Barnechea|P,NaN,NaN
31,v122224342jmch-01,Beetrack,,Boosmap,BOOS-SXKX85,Miguel Morales Marquez,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 13:55:04,2024-02-02,Entregado,Entregado a Domicilio,Lo Barnechea,1,11.0,P,J514|Boosmap|Lo Barnechea|P,NaN,NaN
40,v122181261jmch-01,Beetrack,,Boosmap,BOOS-GYPV24,Anthony Javier Avila Saavedra,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 10:41:54,2024-02-02,Entregado,Entregado a Domicilio,Lo Barnechea,1,9.0,P,J514|Boosmap|Lo Barnechea|P,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55354,v122202265jmch-01,Beetrack,,Boosmap,BOOS-DCLS24,Samuel Iglesias,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 11:18:19,2024-02-02,Entregado,Entregado a Domicilio,None,1,2.0,MP,NaN,NaN,NaN
55358,v122138130jmch-01,Beetrack,,Boosmap,BOOS-SGBP19,Jesús Avendaño,Jumbo,J514,JUMBO LA DEHESA,2024-02-01 14:52:21,2024-02-01,Entregado,Entregado a Domicilio,None,1,4.0,MP,NaN,NaN,NaN
55372,v122242609jmch-01,Beetrack,,Boosmap,BOOS-PPCP47,Jhonathan Farias Linares,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 16:12:43,2024-02-02,Entregado,Entregado a Domicilio,None,1,9.0,P,NaN,NaN,NaN
55376,v122177282jmch-01,Beetrack,,Boosmap,BOOS-BWLW89,Michelle Torrealba Montilva,Jumbo,J514,JUMBO LA DEHESA,2024-02-02 11:39:56,2024-02-02,Entregado,Entregado a Domicilio,None,1,14.0,M,NaN,NaN,NaN


In [75]:
result_pricing_3=result_pricing[result_pricing['orden_compra']=='v122433741jmch-01']
result_pricing_3

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,fecha_compromiso,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost


In [76]:
result_pricing_5=result_pricing
result_pricing_5['commune_status'] = result_pricing_5['comuna_destino'].apply(lambda x: 'Empty' if pd.isna(x) else 'Not Empty')

result_pricing_5[result_pricing_5['commune_status']=='Empty']

,orden_compra,sistema,codigo_conductor,proveedor,patente,nombre_driver,negocio,codigo_tienda,nombre_tienda,fecha_evento,...,nombre_estado,nombre_sub_estado,comuna_destino,on_time,bultos,final_size,provider_key,Cost,final_cost,commune_status
54705,v122191656jmch-01,Beetrack,,Foxer,FOXER-RWVC19,Pedro Gracía,Jumbo,J591,JUMBO TALCA,2024-02-02 10:31:55,...,Entregado,Entregado a Domicilio,None,1,10.0,P,NaN,NaN,NaN,Empty
54706,v121760608jmch-01,Beetrack,,Boosmap,BOOS-PZGP14,Mauricio Gabriel Bustamante Soto,Jumbo,J507,JUMBO PTO. MONTT,2024-02-01 18:10:41,...,Entregado,Entregado a Domicilio,None,1,12.0,P,NaN,NaN,NaN,Empty
54707,v122100293jmch-01,Beetrack,,ZUBALE,ZUBALE-HLRS55,Adonis javier duran quiroz,Jumbo,J659,JUMBO CONCHA Y TORO,2024-02-01 09:53:25,...,Entregado,Entregado a Domicilio,None,1,18.0,M,NaN,NaN,NaN,Empty
54708,v122289567jmch-01,Beetrack,,Boosmap,BOOS-PRKS77,deivy morales,Jumbo,J514,JUMBO LA DEHESA,2024-02-03 09:35:40,...,Entregado,Entregado a Domicilio,None,1,4.0,MP,NaN,NaN,NaN,Empty
54709,v122146903jmch-01,Beetrack,,Boosmap,BOOS-RSJR53,Maximiliano Segovia,Jumbo,J508,JUMBO VIÑA,2024-02-02 18:47:31,...,Entregado,Entregado a Domicilio,None,1,12.0,P,NaN,NaN,NaN,Empty
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55498,v122105326jmch-01,Beetrack,,Foxer,FOXER-RDLB79,Xinaiz Maygualida Azuaje Valecillos,Jumbo,J532,JUMBO CHILLAN,2024-02-01 09:40:15,...,Entregado,Entregado a Domicilio,None,1,5.0,MP,NaN,NaN,NaN,Empty
55499,v122321433jmch-01,Beetrack,,ZUBALE,ZUBALE-FWFL23,Yendler Carrizo,Jumbo,J504,JUMBO RANCAGUA,2024-02-03 18:33:19,...,Entregado,Entregado a Domicilio,None,1,3.0,MP,NaN,NaN,NaN,Empty
55500,v122014558jmch-01,Beetrack,,Foxer,FOXER-RGKJ94,Ramon agustin Ignacio Muñoz Barra,Jumbo,J532,JUMBO CHILLAN,2024-02-01 15:53:24,...,Entregado,Entregado a Domicilio,None,1,8.0,P,NaN,NaN,NaN,Empty
55501,v122121224jmch-01,Beetrack,,ZUBALE,ZUBALE-LLDL40,Hernan Diaz,Jumbo,J504,JUMBO RANCAGUA,2024-02-01 13:28:13,...,Recogido,Entregado a Domicilio,None,1,5.0,MP,NaN,NaN,NaN,Empty


**EJECUTAR 11**

In [77]:
def format_clp(x):
    return '{:,.0f} CLP'.format(x)

def format_thousands(x):
    return '{:,.0f}'.format(x).replace(',', '.')


pivot_table = pd.pivot_table(result_pricing, values='final_cost', index=['codigo_tienda', 'nombre_tienda', 'proveedor'], aggfunc= {'final_cost': ['sum', 'count']})

# Reset index to flatten the pivot table
pivot_table = pivot_table.reset_index()

pivot_table['sum'] = pivot_table['sum'].apply(format_thousands)
pivot_table['count'] = pivot_table['count'].apply(format_thousands)
#pivot_table['final_cost'] = pivot_table['final_cost'].apply(format_clp)



pivot_table


,codigo_tienda,nombre_tienda,proveedor,count,sum
0,J403,JUMBO MAIPU,Boosmap,3.714,21.028.360
1,J410,JUMBO FLORIDA,Kowski Chile,4.518,22.558.080
2,J410,JUMBO FLORIDA,Zubale,0,0
3,J502,JUMBO KENNEDY,Beat,0,0
4,J502,JUMBO KENNEDY,Casos Touch,0,0
5,J502,JUMBO KENNEDY,Touch,4.557,17.664.684
6,J504,JUMBO RANCAGUA,ZUBALE,981,5.777.524
7,J506,JUMBO TEMUCO,Transporte Appelt,600,4.930.804
8,J507,JUMBO PTO. MONTT,Boosmap,488,3.347.696
9,J508,JUMBO VIÑA,Boosmap,2.810,19.018.948


**EJECUTAR 12**

In [78]:
pivot_table2 = pd.pivot_table(result_cruce, values='orden_compra', index=['codigo_tienda', 'nombre_tienda'], aggfunc= {'orden_compra': ['count']})

# Reset index to flatten the pivot table
pivot_table2 = pivot_table2.reset_index()

pivot_table2['count'] = pivot_table2['count'].apply(format_thousands)
#pivot_table['final_cost'] = pivot_table['final_cost'].apply(format_clp)

pivot_table2

,codigo_tienda,nombre_tienda,count
0,J403,JUMBO MAIPU,3.790
1,J410,JUMBO FLORIDA,4.674
2,J414,DARKSTORE LA DEHESA,3
3,J502,JUMBO KENNEDY,4.657
4,J504,JUMBO RANCAGUA,1.062
5,J506,JUMBO TEMUCO,683
6,J507,JUMBO PTO. MONTT,536
7,J508,JUMBO VIÑA,2.973
8,J511,JUMBO COSTANERA,6.061
9,J512,JUMBO LA REINA,5.446


**EJECUTAR 13**

In [80]:
#Create Excel File

import os

excel_file_path = f'{os.getcwd()}_{negocio}_{fecha_compromiso_inicial[:10]}_{fecha_compromiso_final[:10]}.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    pivot_table.to_excel(writer,sheet_name ='Consolidado_Tarifario',index=False, engine='xlsxwriter')
    pivot_table2.to_excel(writer,sheet_name ='Consolidado_Completo',index=False, engine='xlsxwriter')
    result_pricing.to_excel(writer,sheet_name ='BDD_Tarifario',index=False, engine='xlsxwriter')
   
    





**NO USAR**

In [29]:
#Create Excel File (Backup with inner join between cruce and pricing)

excel_file_path = f'/Users/eliassaleh/Desktop/Facturacion/Exportables/Exportable_Delivery_{negocio}_{fecha_compromiso_inicial[:10]}_{fecha_compromiso_final[:10]}.xlsx'

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    pivot_table.to_excel(writer,sheet_name ='Consolidado_Tarifario',index=False, engine='xlsxwriter')
    result_pricing.to_excel(writer,sheet_name ='BDD_Tarifario',index=False, engine='xlsxwriter')
    df_pricing.to_excel(writer,sheet_name ='Tarifario',index=False, engine='xlsxwriter')
    pivot_table2.to_excel(writer,sheet_name ='Consolidado_Completo',index=False, engine='xlsxwriter')
    result_cruce.to_excel(writer,sheet_name ='BDD_Completa',index=False, engine='xlsxwriter')

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/2e/7d/2259318c202f3d17f3fe6438149b3b9e706d1070fe3fcbb28049730bb25c/charset_normalizer-3.3.2-cp312-cp312-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for idna<4,>=2.5 from https://files.pythonhosted.org/packages/c2/e7/a82b05cf63a603df6e68d59ae6a68bf5064484a0718ea5033660af4b54a9/idna-3.6-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/88/75/311454fd3317aefe18415f04568edc20218453b709c63c58b9292c71be17/urllib3-2.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2017.4.17 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e9

In [354]:
import requests

def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"




    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            return "Commune not found" 
    else:
        return "Error fetching data"  # Return error message if request was not successful

# Example usage
latitude = -34.158049468051400	
longitude = -71.470549308961600



commune = get_commune(latitude, longitude)
print("Commune:", commune)







Commune: Las Cabras


In [307]:
import numpy as np
def get_commune(latitude, longitude):
    # Construct the API request URL
    url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Print API response for debugging
        print("API Response:", data)
        
        # Try to extract the commune (comuna) from different keys in the address details
        if 'city' in data['address']:
            return data['address']['city']
        elif 'town' in data['address']:
            return data['address']['town']
        elif 'suburb' in data['address']:
            return data['address']['suburb']
        elif 'village' in data['address']:
            return data['address']['village']
        elif 'hamlet' in data['address']:
            return data['address']['hamlet']
        else:
            print("Address details:", data['address'])  # Print out address details for debugging
            return "Commune not found" 
    else:
        print("Error fetching data")  # Print error message for debugging
        return "Error fetching data"  # Return error message if request was not successful

def get_communes(row):
    if pd.isna(row['commune']):  # Check if commune column is empty
        commune = get_commune(row['latitude'], row['longitude'])
        return commune
    else:
        return row['commune']
    

# Example DataFrame with latitude and longitude columns
data = {
    'latitude': [-39.07890413, -33.4489, -33.4691],
    'longitude': [-71.97376793, -70.6693, -70.642],
    'commune': [np.nan, 'Santiago', np.nan]
}
df_coord = pd.DataFrame(data)


# Apply get_communes function to each row
df_coord['commune'] = df_coord.apply(get_communes, axis=1)



# Print the resulting DataFrame
print("Resulting DataFrame:")
df_coord



#url = f"https://nominatim.openstreetmap.org/reverse.php?lat={latitude}&lon={longitude}&zoom=18&format=jsonv2"

    # Send the request
#response = requests.get(url)
    

#data = response.json()

#latitude= -39.078904
#longitude=- 71.973768

#print("API Response:", data)
#commune = get_commune(latitude, longitude)
#print("Commune:", commune)

API Response: {'place_id': 33550931, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'node', 'osm_id': 6009239545, 'lat': '-39.0641046', 'lon': '-71.97025', 'category': 'place', 'type': 'isolated_dwelling', 'place_rank': 22, 'importance': 0.2000099999999999, 'addresstype': 'isolated_dwelling', 'name': 'Nancahue', 'display_name': 'Nancahue, Cunco, Provincia de Cautín, Región de la Araucanía, Chile', 'address': {'isolated_dwelling': 'Nancahue', 'town': 'Cunco', 'county': 'Provincia de Cautín', 'state': 'Región de la Araucanía', 'ISO3166-2-lvl4': 'CL-AR', 'country': 'Chile', 'country_code': 'cl'}, 'boundingbox': ['-39.0641546', '-39.0640546', '-71.9703000', '-71.9702000']}
API Response: {'place_id': 61385429, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'way', 'osm_id': 110572520, 'lat': '-33.46918361818182', 'lon': '-70.64198729999998', 'category': 'place', 'type': 'house', 'place_rank': 30

,latitude,longitude,commune
0,-39.078904,-71.973768,Cunco
1,-33.448900,-70.669300,Santiago
2,-33.469100,-70.642000,Santiago
